# Glassdoor analysis

In [1]:
# import requests 
# r = requests.get( 
# 'http://api.glassdoor.com/api/api.htmv=1&format=json&t.p=120&t.k=fz6JLNDfgVs&action=employers&q=pharmaceuticals&userip=192.168.43.42&useragent=Mozilla/%2F4.0' 
# ) 
# response_JSON = r.json() 

In [2]:
# !pip install selenium

In [19]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
import numpy as np
from tqdm import tqdm

In [20]:
'''Gathers pharmaceutical companies as a dataframe, scraped from Glassdoor'''    
#Initializing the webdriver
options = webdriver.ChromeOptions()

#Uncomment the line below if you'd like to scrape without a new Chrome window every time.
#options.add_argument('headless')

#Change the path to where chromedriver is in your home folder.
driver = webdriver.Chrome(executable_path="https://github.com/arnaud-dg/Glassdoor_Analysis/blob/main/chromedriver.exe", options=options)
driver.set_window_size(1120, 1000)

url = 'https://www.glassdoor.fr/Avis/index.htm?overall_rating_low=0&page=1&sector=10005&filterType=RATING_OVERALL'
driver.get(url)
companies = []

C:\Users\pharm\AppData\Local\Temp\ipykernel_3312\4071902041.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="https://github.com/arnaud-dg/Glassdoor_Analysis/blob/main/chromedriver.exe", options=options)


In [21]:
# Recherche du nombre d'entreprises référencées
time.sleep(4)
nombre_entreprises = driver.find_element(By.XPATH, "//span[contains(@class,'resultCount css-56kyx5')]").get_attribute('innerHTML')

In [22]:
nombre_entreprises = nombre_entreprises[nombre_entreprises.index('entreprises sur')+24:]
nombre = int(nombre_entreprises[:nombre_entreprises.index('</strong>')])
nombre

539

In [23]:
companies = []
ratings = []
reviews = []
salaries = []
jobs = []
code = []

# Boucle de récupération des noms d'entreprises
while len(companies) < 20:
    time.sleep(4)
    
    entreprises = driver.find_element(By.TAG_NAME, 'h2')
    liste_entreprise = entreprises.find_elements(By.XPATH, '//*[@data-test="employer-short-name"]')
    for e in liste_entreprise:
        companies.append(e.text)
    list_ratings = driver.find_elements(By.XPATH, '//*[@data-test="rating"]')
    for e in list_ratings:
        ratings.append(e.text)
    liste_reviews = driver.find_elements(By.XPATH, '//*[@data-test="cell-Reviews-count"]')
    for e in liste_reviews:
        reviews.append(e.text)
    liste_salaries = driver.find_elements(By.XPATH, '//*[@data-test="cell-Salaries-count"]')
    for e in liste_salaries:
        salaries.append(e.text)
    liste_jobs = driver.find_elements(By.XPATH, '//*[@data-test="cell-Jobs-count"]')
    for e in liste_jobs:
        jobs.append(e.text)
    liste_code = driver.find_elements(By.XPATH, '//*[@data-test="cell-Salaries-url"]')
    for e in liste_code:
        temp = str(e.get_attribute('href'))
        temp = temp.replace("https://www.glassdoor.fr/salaire/", "")
        temp = temp.replace(".htm", "")
        code.append(temp)
        print(temp)
    
    l = driver.find_element(By.XPATH, '//*[@data-test="pagination-next"]')
    # click button
    driver.execute_script("arguments[0].click();", l);

Sanofi-Salaires-E9347
Johnson-and-Johnson-Salaires-E364
IQVIA-Salaires-E8347
GSK-Salaires-E3477
Novartis-Salaires-E6667
Roche-Salaires-E3480
Eurofins-Scientific-Salaires-E25113
Bayer-Salaires-E4245
Thermo-Fisher-Scientific-Salaires-E658
Pfizer-Salaires-E525
AstraZeneca-Salaires-E9214
Boehringer-Ingelheim-Salaires-E10416
AbbVie-Salaires-E649837
Bristol-Myers-Squibb-Salaires-E107
bioM%C3%A9rieux-Salaires-E262158
Merck-Salaires-E438
Servier-Salaires-E41168
MSD-Salaires-E550581
Merck-KGaA-Salaires-E8715
Eli-Lilly-and-Company-Salaires-E223


In [24]:
def k_conversion(value_str):
    
    if "k" in value_str:
        value_str = value_str.replace(" k", "")
        value_str = value_str.replace(",", ".")
        value_int = int(float(value_str) * 1000)
    else:
        value_int = int(value_str)
        
    return value_int

In [25]:
data = list(zip(companies, ratings, reviews, salaries, jobs, code))
df = pd.DataFrame(data, columns = ['Companies', 'Global rating', 'Reviews', 'Salaries', 'Jobs', 'Code'])
for i in ['Reviews', 'Salaries', 'Jobs']:
    df[i] = df[i].apply(k_conversion)
df

,Companies,Global rating,Reviews,Salaries,Jobs,Code
0,Sanofi,"4,0",9300,3100,1400,Sanofi-Salaires-E9347
1,Johnson & Johnson,"4,2",14800,393,1400,Johnson-and-Johnson-Salaires-E364
2,IQVIA,"4,0",12500,374,2500,IQVIA-Salaires-E8347
3,GSK,"4,1",11900,302,2300,GSK-Salaires-E3477
4,Novartis,"4,0",9300,305,2100,Novartis-Salaires-E6667
5,Roche,"3,9",7500,268,1500,Roche-Salaires-E3480
6,Eurofins Scientific,"3,3",4200,582,1800,Eurofins-Scientific-Salaires-E25113
7,Bayer,"4,1",8000,295,1400,Bayer-Salaires-E4245
8,Thermo Fisher Scientific,"3,7",9100,191,5200,Thermo-Fisher-Scientific-Salaires-E658
9,Pfizer,"4,1",8400,158,2600,Pfizer-Salaires-E525


In [26]:
def extract_codesource(codesource, target):
    target = "\"" + target + "\""
    temp = codesource[codesource.index(target)+len(target)+1:]
    return temp[:temp.index(",")]

In [27]:
new_parameters = ["allReviewsCount", "ratedReviewsCount", "overallRating", "reviewCount", "ceoRating", "recommendToFriendRating", 
                 "cultureAndValuesRating", "diversityAndInclusionRating", "careerOpportunitiesRating", "workLifeBalanceRating",
                  "seniorManagementRating", "compensationAndBenefitsRating", "businessOutlookRating"]

In [28]:
for i in new_parameters:
    df[i] = np.nan

In [18]:
for index, row in tqdm(df.iterrows()):
    adresse = row['Code']
    keyword = "Avis"
    adresse = adresse.replace("Salaires", keyword)
    radical = "https://www.glassdoor.fr/"
    url = radical + keyword + "/" + adresse + ".htm?filter.iso3Language=fra"
    driver.get(url)
    time.sleep(4)
    
    code_source = driver.page_source
    for i in new_parameters:
        df[i] = extract_codesource(code_source, i)
        
#     allReviewsCount = extract_codesource(code_source, "allReviewsCount")
#     print(allReviewsCount)
#     ratedReviewsCount = extract_codesource(code_source, "ratedReviewsCount")
#     print(ratedReviewsCount)
#     overallRating = extract_codesource(code_source, "overallRating")
#     print(overallRating)
#     reviewCount = extract_codesource(code_source, "reviewCount")
#     print(reviewCount)
#     ceoRating = extract_codesource(code_source, "ceoRating")
#     print(ceoRating)
#     recommendToFriendRating = extract_codesource(code_source, "recommendToFriendRating")
#     print(recommendToFriendRating)
#     cultureAndValuesRating = extract_codesource(code_source, "cultureAndValuesRating")
#     print(cultureAndValuesRating)
#     diversityAndInclusionRating = extract_codesource(code_source, "diversityAndInclusionRating")
#     print(diversityAndInclusionRating)
#     careerOpportunitiesRating = extract_codesource(code_source, "careerOpportunitiesRating")
#     print(careerOpportunitiesRating)
#     workLifeBalanceRating = extract_codesource(code_source, "workLifeBalanceRating")
#     print(workLifeBalanceRating)
#     seniorManagementRating = extract_codesource(code_source, "seniorManagementRating")
#     print(seniorManagementRating)
#     compensationAndBenefitsRating = extract_codesource(code_source, "compensationAndBenefitsRating")
#     print(compensationAndBenefitsRating)
#     businessOutlookRating = extract_codesource(code_source, "businessOutlookRating")
#     print(businessOutlookRating)
    
    driver.quit()

1it [00:26, 26.09s/it]


MaxRetryError: HTTPConnectionPool(host='localhost', port=51616): Max retries exceeded with url: /session/612adc2495f3c65372cf95de9de8c9d1/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001C0541ED1C0>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée'))